In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Training and Evaluation\n",
    "## WeatherTech Inc. Rainfall Prediction Project\n",
    "\n",
    "This notebook implements and evaluates various classification models for rainfall prediction."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.model_selection import train_test_split, GridSearchCV\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "from sklearn.tree import DecisionTreeClassifier\n",
    "import xgboost as xgb\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "import sys\n",
    "sys.path.append('../src')\n",
    "from model import RainfallPredictor"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Processed Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load the processed dataset\n",
    "df = pd.read_csv('../data/processed/processed_weather_data.csv')\n",
    "\n",
    "# Separate features and target\n",
    "X = df.drop('rainfall', axis=1)\n",
    "y = df['rainfall']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Model Training and Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Define models to test\n",
    "models = {\n",
    "    'Logistic Regression': LogisticRegression(),\n",
    "    'Decision Tree': DecisionTreeClassifier(),\n",
    "    'Random Forest': RandomForestClassifier(),\n",
    "    'XGBoost': xgb.XGBClassifier()\n",
    "}\n",
    "\n",
    "# Train and evaluate each model\n",
    "results = {}\n",
    "for name, model in models.items():\n",
    "    predictor = RainfallPredictor(model)\n",
    "    predictor.train(X, y)\n",
    "    results[name] = predictor.evaluate()\n",
    "    \n",
    "# Display results\n",
    "results_df = pd.DataFrame(results).T\n",
    "print(\"Model Performance Comparison:\")\n",
    "print(results_df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Best Model Optimization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Select best model based on F1 score\n",
    "best_model_name = results_df['f1'].idxmax()\n",
    "print(f\"Best performing model: {best_model_name}\")\n",
    "\n",
    "# Perform hyperparameter tuning\n",
    "if best_model_name == 'Random Forest':\n",
    "    param_grid = {\n",
    "        'n_estimators': [100, 200, 300],\n",
    "        'max_depth': [10, 20, 30, None],\n",
    "        'min_samples_split': [2, 5, 10]\n",
    "    }\n",
    "    \n",
    "    grid_search = GridSearchCV(models[best_model_name], param_grid, cv=5)\n",
    "    grid_search.fit(X, y)\n",
    "    \n",
    "    print(\"\\nBest parameters:\", grid_search.best_params_)\n",
    "    print(\"Best cross-validation score:\", grid_search.best_score_)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Feature Importance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_feature_importance(model, feature_names):\n",
    "    if hasattr(model, 'feature_importances_'):\n",
    "        importances = model.feature_importances_\n",
    "    elif hasattr(model, 'coef_'):\n",
    "        importances = np.abs(model.coef_[0])\n",
    "    else:\n",
    "        return\n",
    "    \n",
    "    feature_imp = pd.DataFrame({'feature': feature_names, 'importance': importances})\n",
    "    feature_imp = feature_imp.sort_values('importance', ascending=False)\n",
    "    \n",
    "    plt.figure(figsize=(10, 6))\n",
    "    sns.barplot(x='importance', y='feature', data=feature_imp)\n",
    "    plt.title('Feature Importance')\n",
    "    plt.show()\n",
    "\n",
    "# Plot feature importance for the best model\n",
    "plot_feature_importance(grid_search.best_estimator_, X.columns)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 }
}